In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
#读取路径
jt_path='E:\jt_labeled.filered.csv'
#read csv 导入数据
data_jt = pd.read_csv(jt_path)
list_data=[data_jt]
#调时间
for data in list_data:   
    data['时间'] = pd.to_datetime(data['时间'],errors='coerce')
    start_time = data['时间'].min()
    data['时间'] = [start_time + pd.Timedelta(milliseconds=5 * i) for i in range(len(data))]
#画蹲底折线图 
plt.rcParams['font.family'] = 'SimHei'
y_data = data_jt['加速度Z(g)']
x_data = data_jt['时间']
plt.figure(figsize=(50, 6))
plt.plot(x_data, y_data)
plt.xlabel("时间")
plt.ylabel("加速度Z(g)")
plt.title("加速度Z(g)变化折线图")
plt.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
x_data = data_jt['时间']
y_data = data_jt['加速度Z(g)']
y1_data=data_jt['加速度X(g)']
y2_data=data_jt['加速度Y(g)']
fig = go.Figure()
fig.add_trace(
    go.Line(x=x_data, y=y_data, name='加速度Z(g)'),
)
fig.add_trace(
    go.Line(x=x_data, y=y1_data, name='加速度X(g)',yaxis="y2"),          #最后一步使该图线对应到轴yaxis2
)
fig.add_trace(
    go.Line(x=x_data, y=y2_data, name='加速度Y(g)',yaxis="y3"),
)
fig.update_layout(
    title_text="加速度变化折线图",
    xaxis_title="时间",
    yaxis=dict(title="加速度Z(g)"),
    yaxis2=dict(title="加速度X(g)",overlaying="y",side="left",tickfont=dict(color="#d62728"),titlefont=dict(color="#d62728")),    #dict是词典，用来调整一堆参数
    yaxis3=dict(title="加速度Y(g)",overlaying="y",side="right",titlefont=dict(color="#9467bd"),tickfont=dict(color="#9467bd")),
    font_family="SimHei"
)
fig.write_html("acceleration_plot.html")
fig.show()

In [ ]:
def find_peaks(data):
    data = np.array(data)                          #找波峰
    peaks = []
    for i in range(1, len(data) - 1):
        if data[i - 1] < data[i] and data[i + 1] < data[i]:
            peaks.append(i)
    return peaks
def find_zeros(data, tolerance=0.1):            #找零点
    data = np.array(data)          
    zeros = []
    for i in range(1, len(data) - 1):
        if abs(data[i]-8)<tolerance:
            zeros.append(i)
    return zeros
def find_troughs(data):
    data = np.array(data)                         #找波谷
    troughs = []
    for i in range(1, len(data) - 1):
        if data[i - 1] > data[i] and data[i + 1] > data[i]:
            troughs.append(i)
    return troughs
z_peaks = find_peaks(y_data)                 
z_troughs = find_troughs(y_data)
z_zeros = find_zeros(y_data)
#设定阈值
extreme_z_peaks = [idx for idx in z_peaks if y_data[idx] > 8.2]
extreme_z_troughs = [idx for idx in z_troughs if y_data[idx] < 7.6]
#提取超常波峰数据
extreme_z_segments = []
extreme_z_time = []
for peak in extreme_z_peaks:
    zero_left = max([z for z in z_zeros if z < peak])
    zero_right = min([z for z in z_zeros if z > peak])
    extreme_z_segments.append(y_data[zero_left:zero_right + 1])
    extreme_z_time.append(x_data[zero_left:zero_right + 1])
#提取超常波谷数据
extreme_z_segments2 = []
extreme_z_time2 = []
for troughs in extreme_z_troughs:
    zero_left = max([z for z in z_zeros if z < troughs])
    zero_right = min([z for z in z_zeros if z > troughs])
    extreme_z_segments2.append(y_data[zero_left:zero_right + 1])
    extreme_z_time2.append(x_data[zero_left:zero_right + 1])
fig = go.Figure()                              #画图
for i, (segment, time) in enumerate(zip(extreme_z_segments, extreme_z_time)):
    fig.add_trace(go.Scatter(x=time, y=segment, mode='lines', name=f'加速度Z(g) 片段 {i + 1}'))
for j, (segment2, time2) in enumerate(zip(extreme_z_segments2, extreme_z_time2)):
    fig.add_trace(go.Scatter(x=time2, y=segment2, mode='lines', name=f'加速度Z(g) 片段 {j + 1}'))
fig.show()

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# 检查 GPU 是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 提取所有完整波形（这里简单假设以零点划分波形）
all_waves = []
for i in range(len(z_zeros) - 1):
    all_waves.append(y_data[z_zeros[i]:z_zeros[i + 1] + 1])

# 标记异常波形
labels = []
for wave in all_waves:
    wave = np.array(wave)
    wave_peaks = find_peaks(wave)
    wave_troughs = find_troughs(wave)
    has_extreme_peak = any(wave[idx] > 8.2 for idx in wave_peaks)
    has_extreme_trough = any(wave[idx] < 7.6 for idx in wave_troughs)
    if has_extreme_peak or has_extreme_trough:
        # 如果是异常波形，将该波形所有数据标记为 1
        wave_labels = [1] * len(wave)
    else:
        # 如果不是异常波形，将该波形所有数据标记为 0
        wave_labels = [0] * len(wave)
    labels.extend(wave_labels)
# 数据处理，保证所有波形长度一致
max_length = max([len(wave) for wave in all_waves])
padded_waves = []
for wave in all_waves:
    padded = np.pad(wave, (0, max_length - len(wave)), mode='constant')
    padded_waves.append(padded)
# 转换为 numpy 数组
X = np.array(padded_waves)
y = np.array(labels)

# 手动划分训练集和测试集
test_size = 0.2
num_samples = len(X)
num_test = int(num_samples * test_size)
# 随机打乱索引
indices = np.random.permutation(num_samples)
test_indices = indices[:num_test]
train_indices = indices[num_test:]

X_train = X[train_indices]
y_train = y[train_indices]
X_test = X[test_indices]
y_test = y[test_indices]

# 添加通道维度
X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)

# 自定义数据集类
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.tensor(data, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# 创建数据集和数据加载器
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 定义 1D CNN 模型
class OneDCNN(nn.Module):
    def __init__(self):
        super(OneDCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(32 * ((max_length - 2) // 2), 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.view(-1, 32 * ((max_length - 2) // 2))
        x = self.fc1(x)
        x = self.sigmoid(x)
        return x

# 初始化模型并移动到 GPU
model = OneDCNN().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    # 使用 tqdm 包装 train_loader 显示训练进度
    train_loader_tqdm = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}')
    for inputs, labels in train_loader_tqdm:
        # 将数据移动到 GPU
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        # 更新 tqdm 进度条显示的信息
        train_loader_tqdm.set_postfix(loss=running_loss / (train_loader_tqdm.n + 1))
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}')
model_path = 'saved_model.pth'      # 保存模型参数
torch.save(model.state_dict(), model_path)
# 评估模型
model.eval()
correct = 0
total = 0
# 使用 tqdm 包装 test_loader 显示评估进度
test_loader_tqdm = tqdm(test_loader, desc='Evaluating')
with torch.no_grad():
    for inputs, labels in test_loader_tqdm:
        # 将数据移动到 GPU
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        predicted = (outputs.squeeze() > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        # 更新 tqdm 进度条显示的信息
        test_loader_tqdm.set_postfix(accuracy=correct / total)

print(f'Test Accuracy: {100 * correct / total}%')

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
jt_path1="C:\\Users\\34980\\Desktop\\大创\\数据\\急停_labeled.csv"
#read csv 导入数据
data_jt1 = pd.read_csv(jt_path1)
list_data1=[data_jt1]
#调时间
for data in list_data1:   
    data['时间'] = pd.to_datetime(data['时间'],errors='coerce')
    start_time = data['时间'].min()
    data['时间'] = [start_time + pd.Timedelta(milliseconds=5 * i) for i in range(len(data))]
#画jt折线图 
y_data = data_jt1['加速度Z(g)']
x_data = data_jt1['时间']
y1_data=data_jt1['加速度X(g)']
y2_data=data_jt1['加速度Y(g)']
fig = go.Figure()
fig.add_trace(
    go.Line(x=x_data, y=y_data, name='加速度Z(g)'),
)
fig.add_trace(
    go.Line(x=x_data, y=y1_data, name='加速度X(g)',yaxis="y2"),          #最后一步使该图线对应到轴yaxis2
)
fig.add_trace(
    go.Line(x=x_data, y=y2_data, name='加速度Y(g)',yaxis="y3"),
)
fig.update_layout(
    title_text="加速度变化折线图",
    xaxis_title="时间",
    yaxis=dict(title="加速度Z(g)"),
    yaxis2=dict(title="加速度X(g)",overlaying="y",side="left",tickfont=dict(color="#d62728"),titlefont=dict(color="#d62728")),    #dict是词典，用来调整一堆参数
    yaxis3=dict(title="加速度Y(g)",overlaying="y",side="right",titlefont=dict(color="#9467bd"),tickfont=dict(color="#9467bd")),
    font_family="SimHei"
)
fig.write_html("acceleration_plot.html")
fig.show()

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

jt_path1="C:\\Users\\34980\\Desktop\\大创\\数据\\急停_labeled.csv"
#read csv 导入数据
data_jt1 = pd.read_csv(jt_path1)
list_data1=[data_jt1]
#调时间
for data in list_data1:   
    data['时间'] = pd.to_datetime(data['时间'],errors='coerce')
    start_time = data['时间'].min()
    data['时间'] = [start_time + pd.Timedelta(milliseconds=5 * i) for i in range(len(data))]
#画蹲低折线图 
y_data = data_jt1['加速度Z(g)']
x_data = data_jt1['时间']
z_zeros = find_zeros(y_data)
# 提取所有完整波形
all_waves = []
for i in range(len(z_zeros) - 1):
    all_waves.append(y_data[z_zeros[i]:z_zeros[i + 1] + 1])
# 标记异常波形
labels = []
for wave in all_waves:
    wave = np.array(wave)
    wave_peaks = find_peaks(wave)
    wave_troughs = find_troughs(wave)
    has_extreme_peak = any(wave[idx] > 9 for idx in wave_peaks)
    has_extreme_trough = any(wave[idx] < 7 for idx in wave_troughs)
    if has_extreme_peak or has_extreme_trough:
        # 如果是异常波形，将该波形所有数据标记为 1
        wave_labels = [1] * len(wave)
    else:
        # 如果不是异常波形，将该波形所有数据标记为 0
        wave_labels = [0] * len(wave)
    labels.extend(wave_labels)
padded_waves = []
for wave in all_waves:
    padded = np.pad(wave, (0, max_length - len(wave)), mode='constant')
    padded_waves.append(padded)

# 转换为 numpy 数组
X = np.array(padded_waves)
y = np.array(labels)

# 添加通道维度
X = np.expand_dims(X, axis=1)

# 自定义数据集类
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.tensor(data, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# 创建数据集和数据加载器
test_dataset = CustomDataset(X, y)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 初始化模型
model = OneDCNN()
# 将模型移动到 GPU 或 CPU
model = model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
# 加载训练好的权重
model.load_state_dict(torch.load(model_path))
# 设置模型为评估模式
model.eval()
# 评估模型
correct = 0
total = 0
# 使用 tqdm 包装 test_loader 显示评估进度
test_loader_tqdm = tqdm(test_loader, desc='Evaluating')
with torch.no_grad():
    for inputs, labels in test_loader_tqdm:
        # 将数据移动到 GPU
        inputs, labels = inputs.to(torch.device("cuda" if torch.cuda.is_available() else "cpu")), labels.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
        outputs = model(inputs)
        predicted = (outputs.squeeze() > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        # 更新 tqdm 进度条显示的信息
        test_loader_tqdm.set_postfix(accuracy=correct / total)

print(f'Test Accuracy: {100 * correct / total}%')